In [ ]:
!pip install -r requirements.txt

In [ ]:
import pandas as pd
import numpy as np
import http.client
import requests
import pandas as pd
import json
import pprint
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
diet_req = pd.read_csv("dietary_req.csv").drop(['Source'],axis=1)
#Dividing by 3 to reflect meal
diet_req = diet_req.set_index('Nutrition')
dietreq_meal=diet_req.div(3)
dietreq_meal.iloc[len(dietreq_meal)-1,:] = dietreq_meal.iloc[len(dietreq_meal)-1,:] * -1

In [ ]:
#Deliverable A: Creating Population Diet Requirement Function
groups = ['M 19-30','F 19-30']
def create_pop(group,activity):
    pop_req = dietreq_meal[group]
    if activity:
        pop_req.loc[0,group]= pop_req.iloc[0,1:3] + 300
    return pop_req
group = create_pop(groups, True).drop(0,axis=0)
group

In [ ]:
price_nutrition = pd.read_csv('PriceData.csv')
price_nutrition=price_nutrition.set_index("Ingredients").drop(['Meal Name','Size','ID','fdcId','Brand'],axis=1)
price_nutrition[0:4]

In [ ]:
from  scipy.optimize import linprog as lp
import warnings

def solve_subsistence_problem2(FoodNutrients,Prices,diet_min,max_weight=None,tol=1e-6):
    p = Prices.dropna()

    # Compile list that we have both prices and nutritional info for; drop if either missing
    use = list(set(p.index.tolist()).intersection(FoodNutrients.columns.tolist()))
    p = p[use]
    Aall = FoodNutrients[p.index].fillna(0)
    Amin = Aall.loc[diet_min.index]
    A = Amin
    b=diet_min

    if max_weight is not None:
        A.loc['Hectograms'] = -1
        b.loc['Hectograms'] = -max_weight
   
    result = lp(p, -A, -b, method='interior-point')

    result.A = A
    result.b = b
    
    if result.success:
        result.diet = pd.Series(result.x,index=p.index)
    else: # No feasible solution?
        result.diet = pd.Series(result.x,index=p.index)*np.nan  

    return result
result = solve_subsistence_problem2(price_nutrition.T,Prices,group['M 19-30'])
result

In [ ]:
from  scipy.optimize import linprog as lp
import numpy as np

def solve_subsistence_problem(FoodNutrients,Prices,diet_min,diet_max,max_weight=None,tol=1e-6):
    """Solve Stigler's Subsistence Cost Problem.

    Inputs:
       - FoodNutrients : A pd.DataFrame with rows corresponding to foods, columns to nutrients.
       - Prices : A pd.Series of prices for different foods
       - diet_min : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing minimum intakes.
       - diet_max : A pd.Series of DRIs, with index corresponding to columns of FoodNutrients,
                    describing maximum intakes.
       - max_weight : Maximum weight (in hectograms) allowed for diet.
       - tol : Solution values smaller than this in absolute value treated as zeros.
       
    """
    p = Prices.apply(lambda x:x.magnitude).dropna()

    # Compile list that we have both prices and nutritional info for; drop if either missing
    use = list(set(p.index.tolist()).intersection(FoodNutrients.columns.tolist()))
    p = p[use]

    # Drop nutritional information for foods we don't know the price of,
    # and replace missing nutrients with zeros.
    Aall = FoodNutrients[p.index].fillna(0)

    # Drop rows of A that we don't have constraints for.
    Amin = Aall.loc[diet_min.index]

    Amax = Aall.loc[diet_max.index]

    # Minimum requirements involve multiplying constraint by -1 to make <=.
    A = pd.concat([Amin,-Amax])

    b = pd.concat([diet_min,-diet_max]) # Note sign change for max constraints

    if max_weight is not None:
        # Add up weights of foods consumed
        A.loc['Hectograms'] = -1
        b.loc['Hectograms'] = -max_weight
        
    # Now solve problem!  (Note that the linear program solver we'll use assumes
    # "less-than-or-equal" constraints.  We can switch back and forth by
    # multiplying $A$ and $b$ by $-1$.)

    result = lp(p, -A, -b, method='interior-point')

    result.A = A
    result.b = b
    
    if result.success:
        result.diet = pd.Series(result.x,index=p.index)
    else: # No feasible solution?
        result.diet = pd.Series(result.x,index=p.index)*np.nan  

    return result
result = solve_subsistence_problem(price_nutrition.T,Prices,group['M 19-30'])

In [ ]:
Prices = price_nutrition['Price'].astype(float)